In [69]:

import pandas as pd
import numpy as np
import os
import sklearn
from sklearn import tree, metrics
from sklearn.model_selection import train_test_split
from google.colab import drive
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import random as rnd
import xgboost as xgb


In [70]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [71]:
maleplayers_df= pd.read_csv('/content/drive/MyDrive/male_players (legacy).csv')

<ipython-input-71-bde49d4aab31>:1: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  maleplayers_df= pd.read_csv('/content/drive/MyDrive/male_players (legacy).csv')


In [72]:
players22_df = pd.read_csv('/content/drive/MyDrive/players_22-1.csv')

<ipython-input-72-2f5f3277ae8b>:1: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  players22_df = pd.read_csv('/content/drive/MyDrive/players_22-1.csv')


In [73]:
maleplayers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161583 entries, 0 to 161582
Columns: 110 entries, player_id to player_face_url
dtypes: float64(18), int64(45), object(47)
memory usage: 135.6+ MB


In [74]:
maleplayers_df.head()

,player_id,player_url,fifa_version,fifa_update,fifa_update_date,short_name,long_name,player_positions,overall,potential,...,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url
0,158023,/player/158023/lionel-messi/150002,15,2,2014-09-18,L. Messi,Lionel Andrés Messi Cuccittini,CF,93,95,...,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3,15+3,https://cdn.sofifa.net/players/158/023/15_120.png
1,20801,/player/20801/c-ronaldo-dos-santos-aveiro/150002,15,2,2014-09-18,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"LW, LM",92,92,...,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,16+3,https://cdn.sofifa.net/players/020/801/15_120.png
2,9014,/player/9014/arjen-robben/150002,15,2,2014-09-18,A. Robben,Arjen Robben,"RM, LM, RW",90,90,...,64+3,64+3,64+3,55+3,46+3,46+3,46+3,55+3,14+3,https://cdn.sofifa.net/players/009/014/15_120.png
3,41236,/player/41236/zlatan-ibrahimovic/150002,15,2,2014-09-18,Z. Ibrahimović,Zlatan Ibrahimović,ST,90,90,...,65+3,65+3,61+3,56+3,55+3,55+3,55+3,56+3,17+3,https://cdn.sofifa.net/players/041/236/15_120.png
4,167495,/player/167495/manuel-neuer/150002,15,2,2014-09-18,M. Neuer,Manuel Peter Neuer,GK,90,90,...,40+3,40+3,36+3,36+3,38+3,38+3,38+3,36+3,87+3,https://cdn.sofifa.net/players/167/495/15_120.png


In [75]:
players22_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Columns: 110 entries, sofifa_id to nation_flag_url
dtypes: float64(16), int64(44), object(50)
memory usage: 16.1+ MB


In [76]:
players22_df.head()

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,...,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,78000000.0,320000.0,34,...,50+3,50+3,50+3,61+3,19+3,https://cdn.sofifa.net/players/158/023/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png
1,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,92,92,119500000.0,270000.0,32,...,60+3,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/22_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1353/60.png,https://cdn.sofifa.net/flags/pl.png
2,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",91,91,45000000.0,270000.0,36,...,53+3,53+3,53+3,60+3,20+3,https://cdn.sofifa.net/players/020/801/22_120.png,https://cdn.sofifa.net/teams/11/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png
3,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",91,91,129000000.0,270000.0,29,...,50+3,50+3,50+3,62+3,20+3,https://cdn.sofifa.net/players/190/871/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,NaN,https://cdn.sofifa.net/flags/br.png
4,192985,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,91,125500000.0,350000.0,30,...,69+3,69+3,69+3,75+3,21+3,https://cdn.sofifa.net/players/192/985/22_120.png,https://cdn.sofifa.net/teams/10/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png


**Data preparation & feature extraction process**

In [77]:

missing_values_male = maleplayers_df.isnull().sum()
missing_values_players = players22_df.isnull().sum()


In [78]:
missing_values_male

player_id           0
player_url          0
fifa_version        0
fifa_update         0
fifa_update_date    0
                   ..
cb                  0
rcb                 0
rb                  0
gk                  0
player_face_url     0
Length: 110, dtype: int64

In [79]:
maleplayers_df.fillna(maleplayers_df.mean,inplace=True)
players22_df.fillna(players22_df.mean,inplace=True)
maleplayers_df = maleplayers_df[maleplayers_df.columns.drop(list(maleplayers_df.filter(regex='url')))]
players22_df = players22_df[players22_df.columns.drop(list(players22_df.filter(regex='url')))]

In [80]:
maleplayers_df

,player_id,fifa_version,fifa_update,fifa_update_date,short_name,long_name,player_positions,overall,potential,value_eur,...,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,158023,15,2,2014-09-18,L. Messi,Lionel Andrés Messi Cuccittini,CF,93,95,100500000.0,...,62+3,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3,15+3
1,20801,15,2,2014-09-18,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"LW, LM",92,92,79000000.0,...,63+3,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,16+3
2,9014,15,2,2014-09-18,A. Robben,Arjen Robben,"RM, LM, RW",90,90,54500000.0,...,64+3,64+3,64+3,64+3,55+3,46+3,46+3,46+3,55+3,14+3
3,41236,15,2,2014-09-18,Z. Ibrahimović,Zlatan Ibrahimović,ST,90,90,52500000.0,...,65+3,65+3,65+3,61+3,56+3,55+3,55+3,55+3,56+3,17+3
4,167495,15,2,2014-09-18,M. Neuer,Manuel Peter Neuer,GK,90,90,63500000.0,...,40+3,40+3,40+3,36+3,36+3,38+3,38+3,38+3,36+3,87+3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161578,269011,23,2,2022-09-26,Zhao Ziye,赵梓业,RW,46,61,110000.0,...,32+2,32+2,32+2,37+2,36+2,31+2,31+2,31+2,36+2,14+2
161579,269019,23,2,2022-09-26,Wu Junjie,吴俊杰,LB,46,58,110000.0,...,41+2,41+2,41+2,43+2,44+2,48+2,48+2,48+2,44+2,15+2
161580,271093,23,2,2022-09-26,K. Barlow,Kailin Barlow,CDM,46,58,110000.0,...,45+2,45+2,45+2,47+2,47+2,46+2,46+2,46+2,47+2,14+2
161581,271555,23,2,2022-09-26,G. Tetteh,Gideon Tetteh,"RW, LW",46,70,150000.0,...,30+2,30+2,30+2,34+2,32+2,28+2,28+2,28+2,32+2,15+2


In [81]:
maleplayers_df.columns

Index(['player_id', 'fifa_version', 'fifa_update', 'fifa_update_date',
       'short_name', 'long_name', 'player_positions', 'overall', 'potential',
       'value_eur',
       ...
       'ldm', 'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk'],
      dtype='object', length=108)

In [82]:
dropped_columns = ['long_name','dob','club_team_id']

In [83]:
maleplayers_df = maleplayers_df.drop(columns = dropped_columns)

players22_df = players22_df.drop(columns=dropped_columns)

In [84]:
maleplayers_df

,player_id,fifa_version,fifa_update,fifa_update_date,short_name,player_positions,overall,potential,value_eur,wage_eur,...,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,158023,15,2,2014-09-18,L. Messi,CF,93,95,100500000.0,550000.0,...,62+3,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3,15+3
1,20801,15,2,2014-09-18,Cristiano Ronaldo,"LW, LM",92,92,79000000.0,375000.0,...,63+3,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,16+3
2,9014,15,2,2014-09-18,A. Robben,"RM, LM, RW",90,90,54500000.0,275000.0,...,64+3,64+3,64+3,64+3,55+3,46+3,46+3,46+3,55+3,14+3
3,41236,15,2,2014-09-18,Z. Ibrahimović,ST,90,90,52500000.0,275000.0,...,65+3,65+3,65+3,61+3,56+3,55+3,55+3,55+3,56+3,17+3
4,167495,15,2,2014-09-18,M. Neuer,GK,90,90,63500000.0,300000.0,...,40+3,40+3,40+3,36+3,36+3,38+3,38+3,38+3,36+3,87+3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161578,269011,23,2,2022-09-26,Zhao Ziye,RW,46,61,110000.0,700.0,...,32+2,32+2,32+2,37+2,36+2,31+2,31+2,31+2,36+2,14+2
161579,269019,23,2,2022-09-26,Wu Junjie,LB,46,58,110000.0,750.0,...,41+2,41+2,41+2,43+2,44+2,48+2,48+2,48+2,44+2,15+2
161580,271093,23,2,2022-09-26,K. Barlow,CDM,46,58,110000.0,500.0,...,45+2,45+2,45+2,47+2,47+2,46+2,46+2,46+2,47+2,14+2
161581,271555,23,2,2022-09-26,G. Tetteh,"RW, LW",46,70,150000.0,500.0,...,30+2,30+2,30+2,34+2,32+2,28+2,28+2,28+2,32+2,15+2


**Feature subsets that show maximum correlation with the dependent variable**

In [85]:
maleplayers_df['total_stats'] = maleplayers_df[['weak_foot','pace','shooting','passing','dribbling','defending','physic']].sum(axis=1, numeric_only=True)
chosen_columns = ['weak_foot', 'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic']

In [86]:
maleplayers_df.loc[:,'total_stats'] = maleplayers_df[chosen_columns].sum(axis=1, numeric_only=True)
players22_df.loc[:,'total_stats'] = players22_df[chosen_columns].sum(axis=1, numeric_only=True)

In [87]:
correlation = maleplayers_df.corr(numeric_only=True)

In [88]:
total_correlation= correlation['overall'].abs().sort_values(ascending=False)

In [89]:
#features with the maximum correlation with the dependent variable
total_correlation

overall                        1.000000
movement_reactions             0.845753
potential                      0.695362
attacking_short_passing        0.499757
mentality_vision               0.493282
international_reputation       0.485234
skill_long_passing             0.483808
power_shot_power               0.480297
age                            0.458390
skill_ball_control             0.457011
skill_curve                    0.415863
power_long_shots               0.409086
mentality_aggression           0.398310
attacking_crossing             0.396833
skill_fk_accuracy              0.387269
attacking_volleys              0.377048
skill_dribbling                0.369827
player_id                      0.368772
power_stamina                  0.360025
power_strength                 0.350786
mentality_positioning          0.350178
attacking_heading_accuracy     0.340029
mentality_penalties            0.334488
skill_moves                    0.330951
attacking_finishing            0.326376


In [90]:
top=10
features = total_correlation[1:top+1].index

In [91]:
features

Index(['movement_reactions', 'potential', 'attacking_short_passing',
       'mentality_vision', 'international_reputation', 'skill_long_passing',
       'power_shot_power', 'age', 'skill_ball_control', 'skill_curve'],
      dtype='object')

**Training a suitable machine learning model with cross-validation that can predict a player's rating**

In [92]:
StandardScaler = StandardScaler()

In [93]:
ML_model= RandomForestRegressor(n_estimators=500,random_state=42)

In [94]:
X = maleplayers_df[features]

y = maleplayers_df['overall']

In [95]:
X

,movement_reactions,potential,attacking_short_passing,mentality_vision,international_reputation,skill_long_passing,power_shot_power,age,skill_ball_control,skill_curve
0,94,95,89,90,5,76,80,27,96,89
1,90,92,82,81,5,72,94,29,92,88
2,89,90,86,84,5,76,86,30,90,85
3,85,90,84,83,5,76,93,32,90,80
4,89,90,42,20,5,41,42,28,31,25
...,...,...,...,...,...,...,...,...,...,...
161578,39,61,38,42,1,33,64,18,45,39
161579,42,58,30,31,1,25,33,19,32,24
161580,50,58,51,40,1,43,51,19,35,30
161581,45,70,40,44,1,26,42,17,43,38


In [96]:
X = StandardScaler.fit_transform(X.copy())

In [97]:
X = pd.DataFrame(X,columns = features)

In [98]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [99]:
model = RandomForestRegressor(n_estimators=30,max_depth=30,min_samples_leaf=2,min_samples_split=5,random_state=42)

In [100]:
model.fit(X_train, y_train)

RandomForestRegressor(max_depth=30, min_samples_leaf=2, min_samples_split=5,
                      n_estimators=30, random_state=42)

In [101]:
y_pred = model.predict(X_test)

In [ ]:
feature_importances = ML_model.feature_importances_

feature_importances

feature_subset = maleplayers_df[features]

feature_subset.head()


In [102]:
#initialize and fit xgbregressor model to the training data.
modelxgb = xgb.XGBRegressor()
modelxgb.fit(X_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [103]:
cross_val_score(modelxgb, X_test, y_test, cv=5)

array([0.95971689, 0.96175456, 0.96080643, 0.96174491, 0.96214538])

In [104]:
y_xgb = modelxgb.predict(X_test)

In [105]:
mean_abs_err = mean_absolute_error(y_test,y_xgb)

In [106]:
mean_sq_err = mean_squared_error(y_test,y_xgb)

In [107]:
r2 = r2_score(y_test,y_xgb)

In [108]:
print(f"Mean Absolute Error: {mean_abs_err}")
print(f"Mean Squared Error: {mean_sq_err}")
print(f"R-squared: {r2}")

Mean Absolute Error: 0.9038236229513477
Mean Squared Error: 1.7272546630892143
R-squared: 0.9651506981637622


In [109]:
parameter_grid= {
    'n_estimators': [10, 20, 30],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}

In [110]:
Grid_Search = GridSearchCV(ML_model,parameter_grid,cv=10,scoring='neg_mean_squared_error')

In [113]:
Grid_Search.fit(X_train,y_train)

GridSearchCV(cv=10,
             estimator=RandomForestRegressor(n_estimators=500, random_state=42),
             param_grid={'max_depth': [10, 20, 30],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [10, 20, 30]},
             scoring='neg_mean_squared_error')

In [158]:
model_pred = np.round(y_pred).astype(int)
model_pred_xgb = np.round(y_xgb).astype(int)

In [159]:
results = pd.DataFrame({
    'Actual': y_test,
    'Predicted': model_pred,
    'Predicted_XGB': model_pred_xgb
})

In [160]:
print(results.head(10))

        Actual  Predicted  Predicted_XGB
9061        63         67             67
154203      64         63             63
149642      68         68             68
136295      63         63             63
74878       69         72             72
34928       75         75             75
76866       67         68             68
516         78         75             76
23371       66         66             66
55920       70         69             68


In [161]:
#choosing the best hyperparameter
best_parameters = Grid_Search.best_params_

In [162]:
best_parameters

{'max_depth': 30,
 'min_samples_leaf': 2,
 'min_samples_split': 10,
 'n_estimators': 30}

In [163]:
from sklearn.linear_model import LinearRegression

In [169]:
# Make sure the columns are aligned
X_train = players22_df[chosen_columns].dropna()
y_train = players22_df.loc[X_train.index, 'overall']

In [170]:

# Initialize the model
lr_model = LinearRegression()

# Fit the model on the training data
lr_model.fit(X_train, y_train)

LinearRegression()

In [171]:
X_val = players22_df[chosen_columns].dropna()
y_val = players22_df.loc[X_val.index, 'overall']

In [174]:
# Make predictions on the validation set
linear_y_val_pred = lr_model.predict(X_val)

In [175]:
# Calculate RMSE on the validation set
lr_val_rmse = np.sqrt(mean_squared_error(y_val, linear_y_val_pred))
print(f"Linear Regression Validation RMSE: {lr_val_rmse}")

Linear Regression Validation RMSE: 3.375829839532424


In [176]:
from sklearn.linear_model import Ridge

In [177]:
# Define the parameter grid for alpha
param_grid = {'alpha': [0.1, 1, 10, 100]}

In [178]:
# Initialize the Ridge model
ridge = Ridge()

In [179]:
# Initialize GridSearchCV
ridge_grid_search = GridSearchCV(estimator=ridge, param_grid=param_grid,
                                 scoring='neg_mean_squared_error', cv=5, n_jobs=-1, verbose=2)

In [180]:
# Fit GridSearchCV to the data
ridge_grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


GridSearchCV(cv=5, estimator=Ridge(), n_jobs=-1,
             param_grid={'alpha': [0.1, 1, 10, 100]},
             scoring='neg_mean_squared_error', verbose=2)

In [181]:
# View the best parameters and the best score
print(f"Best parameters for Ridge: {ridge_grid_search.best_params_}")
print(f"Best RMSE for Ridge: {np.sqrt(-ridge_grid_search.best_score_)}")


Best parameters for Ridge: {'alpha': 0.1}
Best RMSE for Ridge: 4.093674571308319


In [182]:
# Train the final Ridge model using the best parameters
best_ridge_model = ridge_grid_search.best_estimator_

In [183]:
# Fit the model on the entire training data
best_ridge_model.fit(X_train, y_train)

Ridge(alpha=0.1)

In [184]:
# Make predictions on the validation set
y_val_pred_ridge = best_ridge_model.predict(X_val)

In [185]:
# Calculate RMSE on the validation set
ridge_val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred_ridge))
print(f"Ridge Regression Validation RMSE: {ridge_val_rmse}")

Ridge Regression Validation RMSE: 3.375829839532712


### **Testing Using Players from Fifa 2022**

In [123]:
players22_df.head()

,sofifa_id,short_name,player_positions,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,...,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,total_stats
0,158023,L. Messi,"RW, ST, CF",93,93,78000000.0,320000.0,34,170,72,...,64+3,64+3,66+3,61+3,50+3,50+3,50+3,61+3,19+3,4
1,188545,R. Lewandowski,ST,92,92,119500000.0,270000.0,32,185,81,...,66+3,66+3,64+3,61+3,60+3,60+3,60+3,61+3,19+3,4
2,20801,Cristiano Ronaldo,"ST, LW",91,91,45000000.0,270000.0,36,187,83,...,59+3,59+3,63+3,60+3,53+3,53+3,53+3,60+3,20+3,4
3,190871,Neymar Jr,"LW, CAM",91,91,129000000.0,270000.0,29,175,68,...,63+3,63+3,67+3,62+3,50+3,50+3,50+3,62+3,20+3,5
4,192985,K. De Bruyne,"CM, CAM",91,91,125500000.0,350000.0,30,181,70,...,80+3,80+3,79+3,75+3,69+3,69+3,69+3,75+3,21+3,5


In [140]:
#convert chosen columns to numeric
players22_df[chosen_columns] = players22_df[chosen_columns].apply(pd.to_numeric, errors='coerce')

In [141]:
players22_df.loc[:,'total_stats'] = players22_df[chosen_columns].sum(axis=1)

In [149]:
non_numeric_cols = players22_df.select_dtypes(exclude=[np.number]).columns
print("Non-numeric columns:", non_numeric_cols)

Non-numeric columns: Index([], dtype='object')


In [150]:
correlation = players22_df.corr()

In [151]:
total_correlation = correlation['overall'].abs().sort_values(ascending=False)
total_correlation

overall                        1.000000
movement_reactions             0.871823
passing                        0.715001
mentality_composure            0.708867
dribbling                      0.666402
potential                      0.644275
power_shot_power               0.561180
physic                         0.529234
mentality_vision               0.523025
attacking_short_passing        0.516788
skill_long_passing             0.498216
sofifa_id                      0.491523
shooting                       0.489623
international_reputation       0.470647
skill_ball_control             0.460411
age                            0.459451
skill_curve                    0.420423
power_long_shots               0.410010
attacking_crossing             0.401356
mentality_aggression           0.400161
power_stamina                  0.385150
skill_dribbling                0.384351
skill_fk_accuracy              0.381030
attacking_volleys              0.380942
skill_moves                    0.376226


In [152]:
top=10
important_features = total_correlation[1:top+1].index
important_features

Index(['movement_reactions', 'passing', 'mentality_composure', 'dribbling',
       'potential', 'power_shot_power', 'physic', 'mentality_vision',
       'attacking_short_passing', 'skill_long_passing'],
      dtype='object')

In [153]:
X = players22_df[important_features]
y = players22_df['overall']

In [154]:
StandardScaler=StandardScaler

X = StandardScaler.fit_transform(X.copy())
X = pd.DataFrame(X,columns = important_features)

**Testing The Performance of Players with Random Regressor**

In [241]:
rf_model = RandomForestRegressor(n_estimators=30,max_depth=30,min_samples_leaf=2)

In [242]:
rf_model.fit(X_train,y_train)

RandomForestRegressor(max_depth=30, min_samples_leaf=2, n_estimators=30)

In [243]:
X_test = players22_df[chosen_columns].dropna()
y_test = players22_df.loc[X_test.index, 'overall']

In [244]:
#to re-align y_test to ensure it matches the rows of X_test
y_test = y_test.loc[X_test.index]

In [245]:
y_predict = rf_model.predict(X_test)
y_predict

array([90.66600529, 89.9081746 , 90.89338624, ..., 48.18722222,
       48.00444444, 47.91777778])

In [246]:
print(mean_absolute_error(y_test, y_pred))

0.48370334661560477


**Making Predictions with The XGB Regressor Model**

In [247]:
#initialize and fit xgb model
modelxgb = xgb.XGBRegressor()
modelxgb.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [248]:
#to make predictions on the test set
y_predicted = modelxgb.predict(X_test)

In [249]:
print(f"Mean Absolute Error for XGBRegressor: {mean_absolute_error(y_test, y_pred)}")

Mean Absolute Error for XGBRegressor: 0.48370334661560477


**Using Pickle to Save The Model**

In [257]:
import bz2
import pickle
import joblib
import xgboost as xgb
from sklearn.preprocessing import StandardScaler

In [258]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

#save scaler
scaler_filename = '/content/drive/My Drive/Intro to AI/scaler.pkl'
joblib.dump(scaler, scaler_filename)

#train and save model
modelxgb = xgb.XGBRegressor()
modelxgb.fit(X_train_scaled, y_train)
model_filename = '/content/drive/My Drive/Intro to AI/modelxgb.pkl'
joblib.dump(modelxgb, model_filename)


Scaler and model saved successfully.
